# Part I. ETL Pipeline for Pre-Processing the Files

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv
from decimal import Decimal

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 

#print the full length of the rows
print(len(full_data_rows_list))


# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


8056


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


In [5]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

In [6]:
try:
    session.execute("""
        CREATE KEYSPACE IF NOT EXISTS sparkify
        WITH REPLICATION = 
        { 'class': 'SimpleStrategy', 'replication_factor': 1}"""
)
except Exception as e:
    print(e)

In [7]:
#setting the keyspace to sparkify
try:
    session.set_keyspace('sparkify')
except Exception as e:
    print(e)


In [8]:
#writing utility functions to execute queries to ease the way code flows
def execute_query(query):
    try:
        return session.execute(query)
    except Exception as e:
        print(e)
        
def execute_query_with_params(query, params):
    try:
        return session.execute(query, params)
    except Exception as e:
        print(e)
    
def process_csv_file(file, func):
    with open(file, encoding = 'utf-8') as f:
        csvreader = csv.reader(f)
        next(csvreader)
        for line in csvreader:
            func(line)

In [9]:
#creating the session_library table
execute_query("""
    CREATE TABLE IF NOT EXISTS session_library(
        session_id int,
        item_in_session int,
        artist text,
        song text,
        song_length decimal,
        PRIMARY KEY (session_id, item_in_session)
    )
""")


In [10]:
#we create a helper function to help with changing the values in session id to int,
#we also want item in session to be an int
#it is also necessary that the song_length should be a decimal
def insert_to_session_library(line):
    query = "INSERT INTO session_library (session_id, item_in_session, artist, song, song_length)"
    query = query + ' VALUES(%s, %s, %s, %s, %s)'
    queryParameters = (int(line[8]), int(line[3]), line[0], line[9], Decimal(line[5]))
    
    return execute_query_with_params(query, queryParameters)

process_csv_file('event_datafile_new.csv', func=insert_to_session_library)

In [11]:
rows = execute_query('SELECT artist, song, song_length FROM session_library WHERE session_id = 338 AND item_in_session = 4')
for row in rows:
    print(row.artist, row.song, row.song_length)     

Faithless Music Matters (Mark Knight Dub) 495.3073


In [12]:
#setting up the table
execute_query("""
    CREATE TABLE IF NOT EXISTS user_library (
        user_id int, 
        session_id int,
        item_in_session int,
        artist text,
        song text,
        first_name text,
        last_name text,
        PRIMARY KEY ((user_id, session_id), item_in_session)
    )
""")

In [13]:
#setting up the values in the table and getting it ready
def insert_to_user_library(line):
    query = "INSERT INTO user_library (user_id, session_id, item_in_session, artist, song, first_name, last_name)"
    query = query + "VALUES (%s, %s, %s, %s, %s, %s, %s)"
    queryParameters = (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4])
    
    return execute_query_with_params(query, queryParameters)

process_csv_file('event_datafile_new.csv', func=insert_to_user_library)

In [14]:
rows = execute_query("SELECT artist, song, first_name, last_name FROM user_library WHERE user_id = 10 AND session_id = 182 ORDER BY item_in_session")
for row in rows:
    print(row.artist, row.song, row.first_name, row.last_name)

Down To The Bone Keep On Keepin' On Sylvie Cruz
Three Drives Greece 2000 Sylvie Cruz
Sebastien Tellier Kilometer Sylvie Cruz
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


In [15]:
#create the table
execute_query("""
    CREATE TABLE IF NOT EXISTS user_song_library (
        user_id int, 
        song text,
        artist text,
        first_name text,
        last_name text,
        PRIMARY KEY (song, user_id)
    )
""")

                    

In [16]:
#write the helper function
def insert_to_user_song_library(line):
    query = "INSERT INTO user_song_library (user_id, song, artist, first_name, last_name)"
    query = query + "VALUES (%s, %s, %s, %s, %s)"
    queryParameters = (int(line[10]), line[9], line[0], line[1], line[4])
    
    return execute_query_with_params(query, queryParameters)

process_csv_file('event_datafile_new.csv', func=insert_to_user_song_library)

In [17]:
#execuing the query
rows = execute_query("SELECT first_name, last_name FROM user_song_library WHERE song = 'All Hands Against His Own'")
for row in rows:
    print(row.first_name, row.last_name)
                    

Jacqueline Lynch
Tegan Levine
Sara Johnson


In [18]:
execute_query("drop table user_library")
execute_query("drop table session_library")
execute_query("drop table user_song_library")

In [19]:
session.shutdown()
cluster.shutdown()